# Premier modèle K-NN

## Importation des librairies

In [1]:
import pandas as pd
import numpy as np

from create_DataSet import create_DataSet 

## Chargement du DataSet

In [3]:
data = create_DataSet()
data.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Score,Interpretation
0,a,b,c,b,a,1,3.0,2.0,1.0,3.0,10,B
1,a,b,c,a,b,1,3.0,2.0,3.0,1.0,10,B
2,a,b,c,a,b,1,2.0,3.0,1.0,2.0,8,C
3,a,c,a,a,a,2,3.0,2.0,2.0,3.0,10,B
4,a,c,c,b,a,1,NaN,NaN,NaN,3.0,9,C
